In [1]:
import pandas as pd
import numpy as np

In [2]:
%pwd

'/Users/kevinpan/projects/EVC/RI-Voting-Models/pj2_ranked_choice_voting/notebooks'

In [3]:
data = pd.read_csv("../../raw_data/survey_data.csv")

In [4]:
data

,StartDate,EndDate,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,RecipientLastName,RecipientFirstName,RecipientEmail,...,Q11,Q12,Q13,Q14,Q15,Q16,Q17,Q18,Q19,Q20
0,Start Date,End Date,Progress,Duration (in seconds),Finished,Recorded Date,Response ID,Recipient Last Name,Recipient First Name,Recipient Email,...,"Overall, do you feel that Rhode Island is head...",Who did you vote for President,Were you excited to vote for your presidential...,How confident are you that your Rhode Island p...,How confident are you in the ability of local ...,How confident are you that the results of the ...,Where do you primarily get your information ab...,Why did you choose to vote in-person on electi...,Would you support Rhode Island implementing sa...,Would you support Rhode Island implementing ra...
1,11/12/2024 18:19:16,11/12/2024 18:21:01,100,104,True,11/12/2024 18:21:02,R_7LvyIKEDwhcpXXd,NaN,NaN,NaN,...,Somewhat Right,Kamala Harris,Somewhat Excited,Somewhat Confident,Somewhat Confident,Very Confident,NaN,Today was the best option for my schedule,Definitely,I would need more information to make a decision
2,11/12/2024 18:19:22,11/12/2024 18:21:06,100,103,True,11/12/2024 18:21:06,R_3obsi8B2j0maSsH,NaN,NaN,NaN,...,Somewhat Wrong,Kamala Harris,Extremely Excited,Not at All Confident,Somewhat Confident,NaN,NaN,NaN,Definitely,Definitely
3,11/12/2024 18:19:26,11/12/2024 18:21:06,100,99,True,11/12/2024 18:21:06,R_7S003rcKqMsxg0i,NaN,NaN,NaN,...,Somewhat Right,Other,Not Very Excited,Somewhat Confident,Somewhat Confident,Not Very Confident,"Rhode Island Secretary of State's Website,Soci...",I wanted to wait to learn more about the candi...,Definitely,Definitely
4,11/12/2024 18:19:30,11/12/2024 18:21:13,100,103,True,11/12/2024 18:21:14,R_6JRsucMt6YOLuvd,NaN,NaN,NaN,...,Somewhat Right,Kamala Harris,Not Very Excited,Not Very Confident,Very Confident,Somewhat Confident,Friends/Family,"I prefer voting on election day,I forgot to vo...",Definitely,Probably Not
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
554,11/19/2024 17:14:54,11/19/2024 17:15:57,100,63,True,11/19/2024 17:15:58,R_3EGKr6OKUzun8jf,NaN,NaN,NaN,...,Somewhat Wrong,Donald Trump,Extremely Excited,Not Very Confident,Not Very Confident,Not Very Confident,Rhode Island Secretary of State's Website,I prefer voting on election day,I would need more information to make a decision,Probably
555,11/19/2024 17:15:05,11/19/2024 17:16:21,100,76,True,11/19/2024 17:16:22,R_1OJsFnv1rqF4LjL,NaN,NaN,NaN,...,Somewhat Right,Other,Not Very Excited,Somewhat Confident,Somewhat Confident,Somewhat Confident,Rhode Island Secretary of State's Website,I forgot to vote early or by mail,I would need more information to make a decision,Definitely
556,11/19/2024 17:15:19,11/19/2024 17:16:51,100,91,True,11/19/2024 17:16:51,R_10qqeegqzm2aOFb,NaN,NaN,NaN,...,Somewhat Right,Kamala Harris,Not Very Excited,Not at All Confident,Very Confident,Very Confident,Friends/Family,I prefer voting on election day,Definitely,Probably
557,11/19/2024 17:16:04,11/19/2024 17:17:11,100,67,True,11/19/2024 17:17:12,R_7IMmQ5h2aYnvnsE,NaN,NaN,NaN,...,Unsure,Donald Trump,Somewhat Excited,Not Very Confident,Somewhat Confident,Somewhat Confident,"Social Media,Friends/Family",I prefer voting on election day,Definitely,Probably


In [19]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 559 entries, 0 to 558
Data columns (total 37 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   StartDate              559 non-null    object
 1   EndDate                559 non-null    object
 2   Progress               559 non-null    object
 3   Duration (in seconds)  559 non-null    object
 4   Finished               559 non-null    object
 5   RecordedDate           559 non-null    object
 6   ResponseId             559 non-null    object
 7   RecipientLastName      1 non-null      object
 8   RecipientFirstName     1 non-null      object
 9   RecipientEmail         1 non-null      object
 10  ExternalReference      1 non-null      object
 11  LocationLatitude       559 non-null    object
 12  LocationLongitude      559 non-null    object
 13  DistributionChannel    559 non-null    object
 14  UserLanguage           559 non-null    object
 15  ResponseId.1           

In [20]:
data.head()

,StartDate,EndDate,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,RecipientLastName,RecipientFirstName,RecipientEmail,...,Q11,Q12,Q13,Q14,Q15,Q16,Q17,Q18,Q19,Q20
0,Start Date,End Date,Progress,Duration (in seconds),Finished,Recorded Date,Response ID,Recipient Last Name,Recipient First Name,Recipient Email,...,"Overall, do you feel that Rhode Island is head...",Who did you vote for President,Were you excited to vote for your presidential...,How confident are you that your Rhode Island p...,How confident are you in the ability of local ...,How confident are you that the results of the ...,Where do you primarily get your information ab...,Why did you choose to vote in-person on electi...,Would you support Rhode Island implementing sa...,Would you support Rhode Island implementing ra...
1,11/12/2024 18:19:16,11/12/2024 18:21:01,100,104,True,11/12/2024 18:21:02,R_7LvyIKEDwhcpXXd,NaN,NaN,NaN,...,Somewhat Right,Kamala Harris,Somewhat Excited,Somewhat Confident,Somewhat Confident,Very Confident,NaN,Today was the best option for my schedule,Definitely,I would need more information to make a decision
2,11/12/2024 18:19:22,11/12/2024 18:21:06,100,103,True,11/12/2024 18:21:06,R_3obsi8B2j0maSsH,NaN,NaN,NaN,...,Somewhat Wrong,Kamala Harris,Extremely Excited,Not at All Confident,Somewhat Confident,NaN,NaN,NaN,Definitely,Definitely
3,11/12/2024 18:19:26,11/12/2024 18:21:06,100,99,True,11/12/2024 18:21:06,R_7S003rcKqMsxg0i,NaN,NaN,NaN,...,Somewhat Right,Other,Not Very Excited,Somewhat Confident,Somewhat Confident,Not Very Confident,"Rhode Island Secretary of State's Website,Soci...",I wanted to wait to learn more about the candi...,Definitely,Definitely
4,11/12/2024 18:19:30,11/12/2024 18:21:13,100,103,True,11/12/2024 18:21:14,R_6JRsucMt6YOLuvd,NaN,NaN,NaN,...,Somewhat Right,Kamala Harris,Not Very Excited,Not Very Confident,Very Confident,Somewhat Confident,Friends/Family,"I prefer voting on election day,I forgot to vo...",Definitely,Probably Not


In [33]:
%pwd

'/Users/kevinpan/projects/EVC/RI-Voting-Models/pj2_ranked_choice_voting/notebooks'

In [55]:
df = pd.read_csv("temp_2.csv")
df = df.drop(columns=df.columns[0])

In [56]:
df.head()

,Progress,Duration (in seconds),Finished,Q1,Q2,Q3,Q4,Q5,Q6,Unnamed: 22,...,Q11,Q12,Q13,Q14,Q15,Q16,Q17,Q18,Q19,Q20
0,100,104,True,5.0,30-39,Female,Not married,"Black or African American,American Indian or A...",Protestant / Other Christian Denomination,NaN,...,3.0,Kamala Harris,1.0,2.0,2.0,3.0,NaN,Today was the best option for my schedule,4.0,4.0
1,100,103,True,15.0,30-39,Male,Not married,NaN,NaN,NaN,...,1.0,Kamala Harris,3.0,0.0,2.0,3.0,NaN,NaN,4.0,4.0
2,100,99,True,9.0,40-49,Male,Not married,Asian,NaN,NaN,...,3.0,Other,0.0,2.0,2.0,1.0,"Rhode Island Secretary of State's Website,Soci...",I wanted to wait to learn more about the candi...,4.0,4.0
3,100,103,True,1.0,18-24,Male,Not married,Asian,NaN,NaN,...,3.0,Kamala Harris,0.0,1.0,3.0,2.0,Friends/Family,"I prefer voting on election day,I forgot to vo...",4.0,1.0
4,100,120,True,3.0,18-24,Female,Not married,Hispanic or Latino/a,NaN,NaN,...,3.0,Kamala Harris,1.0,2.0,3.0,2.0,"Rhode Island Secretary of State's Website,Soci...","I prefer voting on election day,I forgot to vo...",4.0,3.0


In [57]:
import pandas as pd

q2_mapping = {
    "18-24": 0, "25-29": 1, "30-39": 2, "40-49": 3, "50-64": 4, "65 or older": 5
}
q8_mapping = {
    "Less than $30,000": 0, "$30,000-$49,999": 1, "$50,000-$100,000": 2,
    "$100,000-$199,999": 3, "$200,000 or more": 4
}
q10_mapping = {
    "Unsure": 0, "Somewhat Wrong": 1, "Definitely Wrong": 2, "Somewhat Right": 3, "Definitely Right": 4
}

df["Q2"] = df["Q2"].map(q2_mapping)
df["Q8"] = df["Q8"].map(q8_mapping)
df["Q10"] = df["Q10"].map(q10_mapping)

df["Q4"] = df["Q4"].map({"Married": 1, "Not married": 0})

one_hot_cols = ["Q3", "Q5", "Q6", "Q7", "Q9", "Q12"]
for col in one_hot_cols:
    dummies = pd.get_dummies(df[col], prefix=col, drop_first=True).astype(int) 
    df = df.drop(columns=[col])
    col_index = df.columns.get_loc(col) if col in df.columns else -1
    df = pd.concat([df.iloc[:, :col_index], dummies, df.iloc[:, col_index:]], axis=1)

In [58]:
df

,Progress,Duration (in seconds),Finished,Q1,Q2,Q4,Unnamed: 22,Q8,Q10,Q11,...,Q7_High school graduate / GED,Q7_Less than High School,Q7_Professional degree,Q7_Some college,Q9_Independent,Q9_Other third party,Q9_Republican,Q12_Kamala Harris,Q12_Other,Q20
0,100,104,True,5.0,2.0,0.0,NaN,1.0,0.0,3.0,...,0,0,0,0,0,0,0,1,0,4.0
1,100,103,True,15.0,2.0,0.0,NaN,1.0,4.0,1.0,...,0,0,0,0,0,0,0,1,0,4.0
2,100,99,True,9.0,3.0,0.0,NaN,3.0,2.0,3.0,...,0,0,1,0,1,0,0,0,1,4.0
3,100,103,True,1.0,0.0,0.0,NaN,3.0,3.0,3.0,...,0,0,0,1,1,0,0,1,0,1.0
4,100,120,True,3.0,0.0,0.0,NaN,0.0,2.0,3.0,...,0,0,0,1,0,0,0,1,0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
553,100,63,True,5.0,4.0,1.0,NaN,3.0,2.0,1.0,...,0,0,0,1,0,0,1,0,0,3.0
554,100,76,True,5.0,2.0,1.0,NaN,3.0,2.0,3.0,...,0,0,1,0,1,0,0,0,1,4.0
555,100,91,True,5.0,2.0,1.0,Russian Orthodox,3.0,1.0,3.0,...,0,0,1,0,0,0,1,1,0,3.0
556,100,67,True,5.0,0.0,0.0,NaN,1.0,1.0,2.0,...,0,0,0,1,0,0,1,0,0,3.0


In [59]:
print(list(df.columns))

['Progress', 'Duration (in seconds)', 'Finished', 'Q1', 'Q2', 'Q4', 'Unnamed: 22', 'Q8', 'Q10', 'Q11', 'Q13', 'Q14', 'Q15', 'Q16', 'Q17', 'Q18', 'Q19', 'Q3_Male', 'Q3_Non-binary / Other', 'Q3_Prefer not to say', 'Q5_Asian', 'Q5_Asian,Other', 'Q5_Black or African American', 'Q5_Black or African American,American Indian or Alaska Native', 'Q5_Black or African American,American Indian or Alaska Native,Other', 'Q5_Black or African American,Hispanic or Latino/a', 'Q5_Black or African American,Native Hawaiian or Pacific Islander', 'Q5_Hispanic or Latino/a', 'Q5_Hispanic or Latino/a,American Indian or Alaska Native', 'Q5_Hispanic or Latino/a,Asian', 'Q5_Native Hawaiian or Pacific Islander', 'Q5_Other', 'Q5_White', 'Q5_White,American Indian or Alaska Native', 'Q5_White,Asian', 'Q5_White,Black or African American', 'Q5_White,Black or African American,American Indian or Alaska Native', 'Q5_White,Black or African American,Hispanic or Latino/a', 'Q5_White,Hispanic or Latino/a', 'Q5_White,Hispanic 

In [60]:
df.to_csv("init_cleaned_data_2_25.csv")